In [2]:
import sys
import numpy as np
import pandas as pd
import matplotlib as matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

print('os name:', os.name)
print('python ver:', sys.version)
print('numpy ver:', np.__version__)
print('pandas ver:', pd.__version__)
print('matplotlib ver:', matplotlib.__version__)
print('seaborn ver:', sns.__version__)

os name: nt
python ver: 3.10.8 (tags/v3.10.8:aaaf517, Oct 11 2022, 16:50:30) [MSC v.1933 64 bit (AMD64)]
numpy ver: 1.23.4
pandas ver: 1.5.0
matplotlib ver: 3.6.0
seaborn ver: 0.12.1


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.min_rows', 50)
pd.set_option('display.max_rows', 100)

## 데이터 가공

In [ ]:
# 공통된 컬럼 찾기
set(df_y.columns)&set(df_x.columns)



## 데이터 전처리

In [ ]:
# Target 균등 분포 확인(특히 분류 분석 시)
value_counts(normalize=True)

# Correlation 확인 (높은 상관관계를 갖는 독립변수를 찾아 제거 or 축소(PCA: 주성분분석) 하기 위해)
# 제거 시에는 짝이 되는 두 컬럼 중 Target과 Corr이 더 작은 COL을 삭제
df.corr(numeric_only=True).round(3)
sns.heatmap(df.corr(numeric_only=True))

for idx in range(len(df.corr())):
    s = df.corr().iloc[idx]
    print(s.drop(s.index[idx]).max())

df.iloc[:, :-1].corr()  # 가장 오른쪽에 있는 Target값의 dtype이 object인 경우 제외시키기 위해

# 독립변수와 종속변수 간 선형성 확인을 위해
B= df.corr(numeric_only=True)['Y']
B = B.loc[B.index!='Y']
B

df.loc[:,df.columns != 'Y'].corrwith(df['Y']) #특정 컬럼 제외 후 그 컬럼에 대해 corr 산출

In [ ]:
# Statistically Significant가 나오면 변수로 넣어주자

##### 범주형 - 범주형 두개의 변수는 독립일까? 관계가 있을까?
##### 카이제곱 검정
##### 귀무가설 : 두 변수의 관계는 독립이다
##### 대립가설 : 두 변수의 관계는 독립이 아니다
##### p ≤ 0.05 일때 두 변수는 독립이 아닌 관계가 있는 것
from scipy import stats

def statistical_test(mode, df, cat, target):
    uniques= df[cat].unique()    
    if mode == 't':   # 범주가 1개, 2개 일때 사용 - 범주별 평균의 차이가 유의미한가? (성별에 따른 키 평균)
        group = df.groupby(cat)[target]
        samples = [group.get_group(i) for i in uniques]
        value, p = stats.ttest_ind(*samples)        
    elif mode == 'f':   # 범주가 3개 이상일 때 사용 - 범주별 평균의 차이가 유의미한가? (혈액형에 따른 키 평균)
        group = df.groupby(cat)[target]
        samples = [group.get_group(i) for i in uniques]
        value, p = stats.f_oneway(*samples)
    elif mode == 'c':   # 두 개 범주가 독립성을 갖는지 아닌지? (가사노동의 종류와 구성원은 독립적? 연관성?)
        contingency = pd.crosstab(index=df[cat], columns=df[target])
        value, p, df, expected = stats.chi2_contingency(contingency)
        
    return value, p, 'Statistically Signifcant' if p<=0.05 else 'Statistically Insignifcant'

In [ ]:
# One-Hot Encoding - 명목형 데이터와 더 잘 어울림
df = pd.get_dummies(df)

# Label Encoding - 명목형 데이터(성별, 혈액형, 국가, 지역, 부서)에 사용은 맞지 않음. (왜냐하면 숫자로 바꿔버리면 숫자의 크기가 생겨버려서)
# Label Encoding을 명목형 데이터에 쓰면 오히려 안 좋아지는 결과 나올 수 있음
LabelEncoder().fit_transform()  #preprocessing 모듈 호출 필요

# s.astype('category').cat.codes 사용법


In [27]:
s = pd.Series(list("ABCAABABCC"))
s = s.astype('category').cat.codes  #눈에 보일때는 문자열이지만 실제 저장된 것은 숫자
s.info(memory_usage='deep')
s

<class 'pandas.core.series.Series'>
RangeIndex: 10 entries, 0 to 9
Series name: None
Non-Null Count  Dtype
--------------  -----
10 non-null     int8 
dtypes: int8(1)
memory usage: 138.0 bytes


0    0
1    1
2    2
3    0
4    0
5    1
6    0
7    1
8    2
9    2
dtype: int8

In [28]:
s = s.astype('category') #이걸 쓰면 안에 data set 수정은 못하지만 메모리를 더 적게 씀
s.info(memory_usage='deep')
s
#s[1]="D" #Category로 만들어 버리면 새로 data를 추가 못함

<class 'pandas.core.series.Series'>
RangeIndex: 10 entries, 0 to 9
Series name: None
Non-Null Count  Dtype   
--------------  -----   
10 non-null     category
dtypes: category(1)
memory usage: 270.0 bytes


0    0
1    1
2    2
3    0
4    0
5    1
6    0
7    1
8    2
9    2
dtype: category
Categories (3, int64): [0, 1, 2]

## 데이터 준비

In [ ]:
X = df.iloc[:, :-1]
Y = df.iloc[:, -1]
print(X.shape, Y.shape)  # Row 개수가 동일하지 확인 (Row, Col)

## 스케일링

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.preprocessing import PolynomialFeatures
degree = 3
P = PolynomialFeatures(degree=degree, include_bias=False)
X_poly = P.fit_transform(X)

## 데이터 분할

In [ ]:
from sklearn.model_selection import train_test_split

- test_size = 0.25: 0.0~1.0 테스트 데이터셋 비율
- train_size = None: 0.0~1.0 훈련 데이터셋 비율, test_size를 작성하면 작성하지 않아도 자동 산출
- random_state = None: 정수 값, 난수 발생의 시드(seed) 값
- stratify: y의 지정한 데이터 비율을 유지(층화추출), y가 범주형일 때 사용함

A = train_test_split(X, Y, random_state=0, test_size=0.25, stratify=y)  # Scaler 미사용 시
A = train_test_split(X_scaled, Y, random_state=0, test_size=0.25, stratify=y)  # Scaler 사용 시 X_scaled 확인 필요
A = train_test_split(X_poly, Y, random_state=0, test_size=0.25)  # PolynomialFeatures 사용 시 X_scaled 확인 필요

x_train, x_test, y_train, y_test = A

[x.shape for x in [x_train, x_test, y_train, y_test]]  #데이터 분할된 비율이 제대로 됐는지 확인 목적


## 선형 모델링(분류-범주형)

In [ ]:
# max_iter = 100: Maximum number of iterations taken for the solvers to converge. Tolerance값보다 작으면 max_iter 전에 빠져나옴
Undefitting일 경우 크게, overfitting일 경우 작게 설정
# tol = 0.0001: Tolerance for stopping criteria.
Overfitting일 경우 크게 설정
#C =1: Inverse of regularization strength; must be a positive float, 규제 강도인 alpha와 반대 개념
Overfitting일 경우 작게 설정

from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=3)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=1.0, tol=0.0001, random_state=None, max_iter=100).fit(x_train,y_train)

from sklearn.tree import DecisionTreeClassifier
tree_gini = DecisionTreeClassifier(criterion='gini', random_state=0).fit(x_train, y_train)
tree_ent = DecisionTreeClassifier(criterion='entropy', random_state=0).fit(x_train, y_train)
  # criterion : 'gini'(default), 'entropy'
  # max_depth = None, 트리의 최대 깊이
  # min_sample_split = 2, 노드 내에서 분할이 필요한 최소의 샘플 수
  # max_features = None, 최적의 분할을 찾기 위해 고려할 Feature의 수
  #  - int : Feature의 수, float : 비율, or {'auto', 'sqrt', 'log2'}
  # max_leaf_nodes=None : 최대 말단 노드 개수

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier


## 선형 모델링(회귀-연속형)

In [ ]:
from sklearn.linear_model import LinearRegression  # y1 = a*x1 + b

# Regularization(정칙화) : Overfitting을 해결하기 위해 학습 시 규제 강도 설정
# alpha(규제 강도): Overfitting 발생 시 모델을 단순화(=규제) 시키기 위한 정도, alpha가 너무 크면 과소적합, 너무 작으면 과대적합

from sklearn.linear_model import Lasso  # L1, 가중치의 절댓값의 합이 최소가 되도록 설정, alpha가 커질수록 사용되는 X 종류가 줄어듬, 사전에 Scaling 필요
from sklearn.linear_model import Ridge  # L2, 가중치의 제곱합이 최소가 되도록 설정, 특정 가중치로 쏠리는 형상 방지, 사전에 Scaling 필요

alpha = 1 
model = Ridge(alpha=alpha).fit(x_train, y_train)





from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

## 비선형 모델링

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

## 모델 평가

In [ ]:
model.score(x_train, y_train)
model.score(x_test, y_test)

## 모델 예측

In [ ]:
#실제값
y_test

#예측값
model.predict(x_test)

# 처음 5개만 Test 하고 싶을 때
y_test.iloc[:5].to_numpy() #실제값(0~4번째 컬럼)
model.predict(x_test[:5]) #에측값

# 분류 모델일 경우, 범주 별 예측 확률을 출력해주며 가장 높은 확률을 가진 것이 모델의 예측값으로 출력됨
model.predict_proba(x_test)  

## 성능 평가

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def print_lr_errors(model, X_test, y_test):
    pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, pred)
    mse = mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)  # = mse**0.5
    r2 = r2_score(y_test, pred)
    print(f'MAE:{mae:.3f}, MSE:{mse:.3f}, RMSE:{rmse:.3f}, R2:{r2:.3f}')


print_lr_errors(model, x_train, y_train)
print_lr_errors(model, x_test, y_test)

In [ ]:
from sklearn.model_selection import train_test_split
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=10)
#모델링
def modeling(model,x_train,x_test,y_train,y_test):
    model.fit(x_train,y_train)
    pred = model.predict(x_test)
    metrics(y_test,pred)
#평가 지표
def metrics(y_test,pred):
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test,pred)
    roc_score = roc_auc_score(y_test,pred,average='macro')
    print('정확도 : {0:.2f}, 정밀도 : {1:.2f}, 재현율 : {2:.2f}'.format(accuracy,precision,recall))
    print('f1-score : {0:.2f}, auc : {1:.2f}'.format(f1,roc_score,recall))